In [1]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

urls = ['https://www.remessaonline.com.br/cotacao/cotacao-', 'https://goldrate.com/pt-br/grama-do-ouro-preco-cotacao'
                                                             '-valor/']
url_planilha = "https://drive.google.com/uc?id=1VwVRMIUuEAhjMx66vI91sIS8yqiGhLyP"
output = "planilha.xlsx"


def obter_cotacao(num, moeda_origem):
    url = urls[num] + moeda_origem
    try:
        response = requests.get(url, timeout=60)
    except (requests.ConnectionError, requests.Timeout) as e:
        df.at[index, 'Cotação'] = 0
        time.sleep(1)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    if num == 0:
        rate = soup.find('div', {'class': 'style__Text-sc-1a6mtr6-2'}).text
        rate_int = rate.replace(',', '.').replace('Reais', '').strip()
    else:
        rate = soup.find('em', {'class': 'price-value'}).text
        rate_int = rate.strip()
    return float(rate_int)


df = pd.read_excel(url_planilha, engine='openpyxl')
df['Cotação'] = float('NaN')

for index, row in df.iterrows():
    moeda = row['Moeda'].lower()
    if moeda == 'ouro':
        cotacao_moeda = obter_cotacao(1, '')
    else:
        cotacao_moeda = obter_cotacao(0, moeda)
    df.at[index, 'Cotação'] = cotacao_moeda
    df.at[index, 'Preço de Compra'] = (cotacao_moeda * df.at[index, 'Preço Original']).round(2)
    df.at[index, 'Preço de Venda'] = (df.at[index, 'Margem'.replace(',', '.')] * df.at[index, 'Preço de Compra']).round(
        2)

df.to_excel(output, index=False)

print(df)


         Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar     4.82          4819.95    1.40   
1   Carro Renault         4500.00   Euro     5.27         23715.00    2.00   
2   Notebook Dell          899.99  Dólar     4.82          4337.95    1.70   
3          IPhone          799.00  Dólar     4.82          3851.18    1.70   
4      Carro Fiat         3000.00   Euro     5.27         15810.00    1.90   
5  Celular Xiaomi          480.48  Dólar     4.82          2315.91    2.00   
6        Joia 20g           20.00   Ouro   303.72          6074.40    1.15   

   Preço de Venda  
0         6747.93  
1        47430.00  
2         7374.52  
3         6547.01  
4        30039.00  
5         4631.82  
6         6985.56  
